In [ ]:
# JM, 30 Dec 2022
#
# plot and make movies with the vertical component of relative vorticity

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import xarray as xr

import cmocean

import datetime

from matplotlib.colors import Normalize, LinearSegmentedColormap
from matplotlib import cm
from matplotlib.colorbar import ColorbarBase
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colorbar import make_axes
import matplotlib.ticker as mticker

from pyCDFTOOLS.cdfcurl import *
from pyCDFTOOLS.draw_clock import *

# define some defaults\n",
plt.rcParams["font.family"] = "DejaVu Serif"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["mathtext.rm"] = "serif"
plt.rcParams["image.cmap"] = "RdBu_r" # \\\"*_r\\\" is reverse of standard colour\\n\",\n",
plt.rcParams["axes.formatter.limits"] = [-4, 4]
plt.rcParams["font.size"] = 12.0

# for vorticity sesmic is good, RdBu_r is ok too
# with cmocean, curl and balance are probably ok
vmin, vmax = -2e-5, 2e-5
misc_opts = {"levels" : np.linspace(vmin, vmax, 41),
             "cmap"   : cmocean.cm.curl,
             "extend" : "both"}

In [ ]:
# plot the vorticity time snapshots over the SAME experiments (in prep for later)

# initial load
data_dir = "../../UNAGI/EXP_R025/eye_candy/no_split_gm0000/OUTPUTS/"
fileU = "UNAGI_5d_04910101_05001230_surf_U.nc"
fileV = fileU.replace("U.nc", "V.nc")

# load the time vector (default should be seconds since 1900-01-01)
data = xr.open_dataset(data_dir + fileU, decode_times=False)
time_vec = data["time_centered"][:]
data.close()
t0 = time_vec[0]                          # reference time
time_vec = (time_vec - t0) / (3600 * 24)  # this is in days

yyyy_ref = 491 # start year

# cycle over fime

fig, axs = plt.subplots(2, 2, figsize=(14, 4))

ax_clock = plt.axes([0.363, 0.36, .30, .30], polar = True)

for kt in range(len(time_vec)):
    kwargs = {"kz"     : 0,
              "kt"     : kt,
              "lprint" : False,
              "lperio" : True,
              "loverf" : False}

    data_dir = "../../UNAGI/EXP_R025/eye_candy/no_split_gm0000/OUTPUTS/"
    lonT, latT, curlu, opt_dic = cdfcurl(data_dir, fileU, "ssu", fileV, "ssv", **kwargs)
    
    for i in range(2):
        for j in range(2):
            # chop out the periodic points because it kills the plotting for some reason
            cs = axs[i][j].contourf(lonT[:, 1:-1], latT[:, 1:-1], curlu[:, 1:-1], **misc_opts)
            axs[i][j].set_aspect('equal')
            axs[i][j].set_ylim(0, 2400)
            axs[i][j].set_xlim(0, 9000)
            axs[i][j].set_xticklabels("")
            axs[i][j].set_yticklabels("")

    # formatting of subplots        
#     axs[0][0].set_ylabel(r"$y$ ($\mathrm{km}$)")
#     axs[0][0].set_xticklabels("")
#     axs[0][1].set_xticklabels("")
#     axs[0][1].set_yticklabels("")
#     axs[1][0].set_xlabel(r"$x$ ($\mathrm{km}$)")
#     axs[1][0].set_ylabel(r"$y$ ($\mathrm{km}$)")
#     axs[1][1].set_xlabel(r"$x$ ($\mathrm{km}$)")
#     axs[1][1].set_yticklabels("")

    # draw clock (days here don't exceed 26 so draw_clock won't be complaining)
    yyyymmdd = "%.4d%.2d%.2d" % (int(time_vec[kt] / 360) + yyyy_ref,  # year starting at zero
                                 int(time_vec[kt] / 30 % 12) + 1,     # month starting at one
                                 (time_vec[kt] % 30) + 1)             # day starting at one
    draw_clock(yyyymmdd, ax = ax_clock, fontsize = 10, 
               clock_color = cmocean.cm.curl(135), progress_color = cmocean.cm.curl_r)

    # colorbar: create the axes by grabbing the bounding boxes of the individual plot axes
    #           only create this once
#     if kt == 0:
#         pos_bot = axs[1][1].get_position()
#         pos_top = axs[0][1].get_position()
#         ax_cb = fig.add_axes([pos_bot.x1 + 0.01,
#                               pos_bot.y0,
#                               0.005,
#                               pos_top.y1 - pos_bot.y0])
#         norm = Normalize(vmin = vmin, vmax = vmax)
#         colors = misc_opts["cmap"](np.linspace(0, 1, misc_opts["cmap"].N))
#         cmap2 = LinearSegmentedColormap.from_list('dummy', colors)
#     cb = ColorbarBase(ax_cb, cmap = cmap2, norm = norm)
#     cb.ax.set_ylabel(r"$\zeta\ (\mathrm{s}^{-1})$")

    # save file
    filename = f"../../UNAGI/EXP_R025/eye_candy/video/UNAGI_R025_xi_{kt:04d}.png"
    fig.savefig(filename, dpi = 100, bbox_inches="tight")

    # clear axes otherwise filesize starts accumulating via stacking of images (instead of overwritting)
    for i in range(2):
        for j in range(2):
            axs[i][j].clear()
#     ax_cb.clear()
    ax_clock.clear()

In [ ]:
# plot the vorticity time snapshots over varying experiments

exp_list = ["no_split_gm0000", "no_split_gm0500", "no_split_alp0060_lam80", "split_100km_alp0060_lam80"]

# initial load
data_dir = f"../../UNAGI/EXP_R025/eye_candy/{exp_list[0]}/OUTPUTS/"
fileU = "UNAGI_5d_05010101_05101230_surf_U.nc"
fileV = fileU.replace("U.nc", "V.nc")

# load the time vector (default should be seconds since 1900-01-01)
data = xr.open_dataset(data_dir + fileU, decode_times=False)
time_vec = data["time_centered"][:]
data.close()
t0 = time_vec[0]                          # reference time
time_vec = (time_vec - t0) / (3600 * 24)  # this is in days

yyyy_ref = 501 # start year

# cycle over fime

fig, axs = plt.subplots(2, 2, figsize=(14, 4))

ax_clock = plt.axes([0.363, 0.36, .30, .30], polar = True)

for kt in range(len(time_vec)):
    kwargs = {"kz"     : 0,
              "kt"     : kt,
              "lprint" : False,
              "lperio" : True,
              "loverf" : False}

    ind = 0
    for i in range(2):
        for j in range(2):

            data_dir = f"../../UNAGI/EXP_R025/eye_candy/{exp_list[ind]}/OUTPUTS/"
            lonT, latT, curlu, opt_dic = cdfcurl(data_dir, fileU, "ssu", fileV, "ssv", **kwargs)

            # chop out the periodic points because it kills the plotting for some reason
            cs = axs[i][j].contourf(lonT[:, 1:-1], latT[:, 1:-1], curlu[:, 1:-1], **misc_opts)
            axs[i][j].set_aspect('equal')
            axs[i][j].set_ylim(0, 2400)
            axs[i][j].set_xlim(0, 9000)
            axs[i][j].set_xticklabels("")
            axs[i][j].set_yticklabels("")

            ind += 1

    # formatting of subplots        
#     axs[0][0].set_ylabel(r"$y$ ($\mathrm{km}$)")
#     axs[0][0].set_xticklabels("")
#     axs[0][1].set_xticklabels("")
#     axs[0][1].set_yticklabels("")
#     axs[1][0].set_xlabel(r"$x$ ($\mathrm{km}$)")
#     axs[1][0].set_ylabel(r"$y$ ($\mathrm{km}$)")
#     axs[1][1].set_xlabel(r"$x$ ($\mathrm{km}$)")
#     axs[1][1].set_yticklabels("")

    # draw clock (days here don't exceed 26 so draw_clock won't be complaining)
    yyyymmdd = "%.4d%.2d%.2d" % (int(time_vec[kt] / 360) + yyyy_ref,  # year starting at zero
                                 int(time_vec[kt] / 30 % 12) + 1,     # month starting at one
                                 (time_vec[kt] % 30) + 1)             # day starting at one
    draw_clock(yyyymmdd, ax = ax_clock, fontsize = 10, 
               clock_color = cmocean.cm.curl(135), progress_color = cmocean.cm.curl_r)

    # colorbar: create the axes by grabbing the bounding boxes of the individual plot axes
    #           only create this once
#     if kt == 0:
#         pos_bot = axs[1][1].get_position()
#         pos_top = axs[0][1].get_position()
#         ax_cb = fig.add_axes([pos_bot.x1 + 0.01,
#                               pos_bot.y0,
#                               0.005,
#                               pos_top.y1 - pos_bot.y0])
#         norm = Normalize(vmin = vmin, vmax = vmax)
#         colors = misc_opts["cmap"](np.linspace(0, 1, misc_opts["cmap"].N))
#         cmap2 = LinearSegmentedColormap.from_list('dummy', colors)
#     cb = ColorbarBase(ax_cb, cmap = cmap2, norm = norm)
#     cb.ax.set_ylabel(r"$\zeta\ (\mathrm{s}^{-1})$")

    # save file
    filename = f"../../UNAGI/EXP_R025/eye_candy/video/UNAGI_R010_xi_{kt+len(time_vec):04d}.png"
    fig.savefig(filename, dpi = 100, bbox_inches="tight")

    # clear axes otherwise filesize starts accumulating via stacking of images (instead of overwritting)
    for i in range(2):
        for j in range(2):
            axs[i][j].clear()
#     ax_cb.clear()
    ax_clock.clear()

In [ ]:
len(time_vec)